# License

    Jupyter notebook for accessing CUDA
    Copyright (C) 2018 Andre.Brodtkorb@ifi.uio.no, changed in October by André Brodtkorb

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [51]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray
import pycuda.driver as cuda_driver

from matplotlib import pyplot as plt

import IPythonMagic

In [52]:
#For timing
from Timer import Timer

#For logging
import logging

In [53]:
import pytest
from ipytest import run_pytest, clean_tests

In [54]:
%setup_logging
%cuda_context_handler context

Global logger already initialized!
Registering context in user workspace
Context already registered! Ignoring


In [58]:
cuda_kernel = """
__global__ void matrixVectorKernel(float* c, float* A, float* b, int a_rows, int a_cols) {
    unsigned int j = blockIdx.x*blockDim.x + threadIdx.x; //more efficient threads aligned horizontally
    
    //Out of bounds check
    if (j > a_rows) {
        return;
    }
    
    //Compute inner product of row of A with column of B
    float sum = 0.0f;
    for (int i=0; i<a_cols; ++i) {
        unsigned int k = j*a_cols + i;
        sum += A[k] * b[i];
    }
    
    //Write to global memory
    c[j] = sum;
}
"""
module = cuda_compiler.SourceModule(cuda_kernel)
kernel = module.get_function("matrixVectorKernel");
#matrixVectorKernel(float* c, float* A, float* b, int a_rows, int a_cols)
#The arguments are Pointer,Pointer,Pointer, Int,Int
kernel.prepare("PPPii")

LogicError: cuModuleLoadDataEx failed: an illegal memory access was encountered - 

In [57]:
def gpuMatrixVector(a, b):
    #Create an operation of the GPU
    stream = cuda_driver.Stream()
    #context.synchronize()
    
    #Upload data to the device
    #NOTE: We need to make sure that a=(a_rows, a_columns)
    # and that b=(a_colmuns, 1) (column vector)
    # and that c=(a_rows, 1)
    with Timer("Data allocation") as t:
        a_g = GPUArray(a.shape, np.float32)
        b_g = GPUArray(b.shape, np.float32)
        #Allocate output data
        c_g = GPUArray(a.shape[0], np.float32)
        context.synchronize()
        
    with Timer("A upload") as t:
        #a_g.set(a)
        #context.synchronize()
        a_g.set_async(a, stream=stream)
        
    with Timer("B upload") as t:
        #b_g.set(b)
        #context.synchronize()
        b_g.set_async(b, stram,stream)
    
    #NOTE: We need to change this so that the grid*block is x = 1, y = number of rows in A
    block_size = (32, 1, 1) #These need to be [x, y, z]
    grid_size = (int(np.ceil(a.shape[0] / 1)), 1, 1)

    print("Block size is " + str(block_size))
    print("Grid size is " + str(grid_size))
    
    #Execute program on device
    with Timer("Kernel execution") as t:
        #kernel(c_g, a_g, b_g, np.int32(a.shape[0]), np.int32(a.shape[1]), block=block_size, grid=grid_size)
        for i in range(100):
            kernel.prepared_async_call(grid_size, block_size, stream, \
                                   c_g.gpudata, a_g.gpudata, b_g.gpudata, np.int32(a.shape[0]), np.int32(a.shape[1]), block=block_size, grid=grid_size)
        #context.synchronize()
        
    #Copy data from device to host
    with Timer("Allocate c") as t:
        c = np.empty((a.shape[0],1), dtype=np.float32)
    
    #insert timing function
    with Timer("Download", logging.INFO) as t:
        c_g.get(c)
        context.synchronize() #wait to synchronize GPU and CAPU
    
    #Return our computed matrix-vector product
    return c #To skip printing of c

In [49]:
nx = np.array([200, 400, 800, 1600, 3200, 6400, 8000])
ny = nx

print(nx, ny)

[ 200  400  800 1600 3200 6400 8000] [ 200  400  800 1600 3200 6400 8000]


In [50]:
#times = np.empty_like(nx)
#for i in range(len(nx)):
   # print("Nx = " + str(nx[i]), flush=True)
    #Size of our test
    
    
test_size = (2048, 2048)

    #Create test input / output data
with Timer("Create test data") as t:
    a = np.random.random(test_size).astype(np.float32)
    b = np.random.random((test_size[1], 1)).astype(np.float32)

with Timer('Run whole function', logging.INFO) as t:
    c = gpuMatrixVector(a, b)
#times[i] = t.msecs
    
#print(nx*ny)
#print(times)


fig = plt.figure()
plt.subplot(1,3,1)
plt.imshow(a)
plt.subplot(1,3,2)
plt.imshow(b)
plt.subplot(1,3,3)
plt.imshow(c)
fig.show()

Create test data: 73.431969 ms
Run whole function: 0.063896 ms


LogicError: cuStreamCreate failed: an illegal memory access was encountered

In [15]:
plt.figure()
#plt.plot(nx*ny, times, 'x')

In [12]:
#Compute reference using Numpy
c_ref = np.dot(a, b)

#Sum of absolute differences
sad = np.sum(np.abs(c - c_ref))

#Print result
# print("C   = ", c)
# print("Ref = ", c_ref)
print("Sad = %.30f" % sad)
print("Per element error: " + str(sad / test_size[1]))

Sad = 0.537384033203125000000000000000
Per element error: 0.0002623945474624634


In [16]:
clean_tests() #this initializes the pytest framework

def test_gpuMatrixVector():
    #Let us test a matrix of size 1x1
    a = np.ones((1,1), dtype=np.float32)
    b = 2*np.ones((1,1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(2.0) #make sure it is approximate otherwise it will fail
    
    #Test that the inner product works
    a = np.ones((1,2), dtype=np.float32)
    b = 2*np.ones((2,1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(4.0)
    
    #Test a general matrix
    test_size = (4, 3)
    a = np.random.random(test_size).astype(np.float32)
    b = np.random.random((test_size[1], 1)).astype(np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(np.dot(a, b), rel = 1e-3)
    
run_pytest(filename='MatrixVectorTesting.ipynb', pytest_options=['-vvv']) #vvv: be really really verbose! We need info

============================= test session starts ==============================
platform linux -- Python 3.6.6, pytest-3.8.2, py-1.6.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /home/ubuntu/jupyter_notebooks/Fabio/MilanoGPU2018/notebooks, inifile:
collecting ... collected 1 item

MatrixVectorTesting.py::test_gpuMatrixVector <- <ipython-input-16-21c36f96d5a4> 

Memory copy time: 0.162601 ms
Memory copy time: 0.143051 ms
Memory copy time: 0.140905 ms


PASSED [100%]

=============================== warnings summary ===============================
/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: get() between arrays of different shape is deprecated and will be removed in PyCUDA 2017.x
/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: get() between arrays of different shape is deprecated and will be removed in PyCUDA 2017.x
/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: get() between arrays of different shape is deprecated and will be removed in PyCUDA 2017.x

-- Docs: https://docs.pytest.org/en/latest/warnings.html
===================== 1 passed, 3 warnings in 0.03 seconds =====================


0